In [69]:
import pandas as pd
import tensorflow
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences



In [70]:
file_path = 'diagnoses_data.csv'
df = pd.read_csv(file_path)
df.head()

,City,State,Aspergillus flavus/oryzae,Aspergillus fumigatus,Aspergillus niger,Aspergillus ochraceus,Aspergillus penicillioides,Aspergillus restrictus,Aspergillus sclerotiorum,Aspergillus sydowii,...,Resipiratory,Digestive,Circulatory,Skin,Brain,Nervous,Urinary,Immune,Reproductive,Diagnoses
0,Mountain Brook,Alabama,-0.104428,-0.073751,-0.337412,-0.055444,-0.067114,-0.158983,-0.0963,-0.084850,...,0,1,1,1,1,1,1,0,1,"Hyperthyroid/Hypothyroid, IBS, PCOS"
1,Wasilla,Alaska,-0.104428,-0.073751,-0.339540,-0.055444,-0.066880,-0.198118,-0.0963,-0.084850,...,1,1,1,1,1,1,0,1,1,0
2,Soldotna,Alaska,-0.104428,-0.072315,-0.349116,0.052017,-0.067114,-0.176772,-0.0963,-0.078989,...,1,1,1,1,1,1,1,1,1,"CIRS, MCAS"
3,Soldotna,Alaska,-0.104428,-0.073751,-0.352308,-0.055444,-0.067114,-0.198118,-0.0963,-0.084850,...,0,0,0,0,0,0,0,0,0,"Auto-Immune Disease, Lyme Disease, Hyperthyroi..."
4,Anchorage,Alaska,0.392251,-0.071357,-0.318261,-0.055444,-0.067114,-0.176772,-0.0963,-0.084850,...,0,1,1,1,1,1,0,0,0,"CIRS, Hyperthyroid/Hypothyroid, PCOS, Candidia..."


In [71]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Diagnoses'])
df['Diagnoses'] = tokenizer.texts_to_sequences(df['Diagnoses'])

In [78]:
max_sequence_length = 140
numerical_feature_columns = list(range(2, 39))
numerical_features = df.iloc[:, numerical_feature_columns].values
text_features = text_features = pad_sequences(df['Diagnoses'], maxlen=max_sequence_length)
target_diagnoses = df['Diagnoses'].to_numpy()

In [79]:
# Define the neural network architecture
model = Sequential()
model.add(Input(shape=(numerical_features.shape[1] + text_features.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Change this line to have 1 output node for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [80]:
mlb = MultiLabelBinarizer()
target_diagnoses = mlb.fit_transform(df['Diagnoses'])

In [81]:
# Train-test split
X_train_num, X_test_num, X_train_text, X_test_text, y_train, y_test = train_test_split(
    numerical_features, text_features, target_diagnoses, test_size=0.2, random_state=42
)


In [83]:
# Train-test split
X_train_num, X_test_num, X_train_text, X_test_text, y_train, y_test = train_test_split(
    numerical_features, text_features, target_diagnoses, test_size=0.2, random_state=42
)

# Ensure that X and y have the same number of rows
num_samples = min(X_train_num.shape[0], X_train_text.shape[0], y_train.shape[0])
X_train_num, X_train_text, y_train = X_train_num[:num_samples], X_train_text[:num_samples], y_train[:num_samples]

# Ensure the same for the test set if needed
num_samples_test = min(X_test_num.shape[0], X_test_text.shape[0], y_test.shape[0])
X_test_num, X_test_text, y_test = X_test_num[:num_samples_test], X_test_text[:num_samples_test], y_test[:num_samples_test]

# Train the model
model.fit([X_train_num, X_train_text], y_train, epochs=10, batch_size=32, validation_data=([X_test_num, X_test_text], y_test))


Epoch 1/10


ValueError: in user code:

    File "/home/codespace/.python/current/lib/python3.10/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/home/codespace/.python/current/lib/python3.10/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/codespace/.python/current/lib/python3.10/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/home/codespace/.python/current/lib/python3.10/site-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/home/codespace/.python/current/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/codespace/.python/current/lib/python3.10/site-p

In [82]:

model.fit([X_train_num, X_train_text], target_diagnoses, epochs=10, batch_size=32, validation_data=([X_test_num, X_test_text], y_test))


ValueError: Data cardinality is ambiguous:
  x sizes: 464, 464
  y sizes: 581
Make sure all arrays contain the same number of samples.